In [1]:
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
import pandas as pd

In [3]:
df = pd.read_excel("../Articles/MediaBiasGroup/labeled_dataset.xlsx")

C:\Users\dario\miniconda3\envs\similarNews\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
df = df[df["Label_bias"] != 'No agreement']

In [5]:
df.loc[df["Label_bias"]=="Non-biased", ["Label_bias"]] = 0
df.loc[df["Label_bias"]=="Biased", ["Label_bias"]] = 1

In [6]:
corpus = df["sentence"]

In [7]:
import nltk
import gensim
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def remove_stopwords(texts):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

stopword_corpus = remove_stopwords(corpus)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

cleaned_corpus = lemmatization(stopword_corpus)

In [9]:
de_tokenized = [' '.join(sentence) for sentence in cleaned_corpus]

In [10]:
df["cleaned_text"] = de_tokenized

In [10]:
train=df.sample(frac=0.8,random_state=200)
test=df.drop(train.index)

In [11]:
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

hub_layer = hub.KerasLayer(module_url, input_shape=[], 
                           dtype=tf.string, trainable=True)


In [12]:
model = tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(40, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.1)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(40, activation = 'relu',kernel_regularizer=tf.keras.regularizers.l2(0.1)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(1, activation = 'relu',kernel_regularizer=tf.keras.regularizers.l2(0.1)),
])

# Compile model:
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['binary_accuracy'])

In [13]:
model.fit(tf.convert_to_tensor(train["cleaned_text"]), tf.convert_to_tensor(train["Label_bias"].astype("category")), epochs=10)

Epoch 1/10
39/39 [==============================] - 11s 123ms/step - loss: 9.5677 - binary_accuracy: 0.5423
Epoch 2/10
39/39 [==============================] - 5s 125ms/step - loss: 4.8691 - binary_accuracy: 0.5939
Epoch 3/10
39/39 [==============================] - 5s 124ms/step - loss: 2.7467 - binary_accuracy: 0.7301
Epoch 4/10
39/39 [==============================] - 5s 124ms/step - loss: 1.6172 - binary_accuracy: 0.9484
Epoch 5/10
39/39 [==============================] - 5s 125ms/step - loss: 1.0365 - binary_accuracy: 0.9919
Epoch 6/10
39/39 [==============================] - 5s 125ms/step - loss: 0.7434 - binary_accuracy: 0.9960
Epoch 7/10
39/39 [==============================] - 5s 123ms/step - loss: 0.5651 - binary_accuracy: 0.9992
Epoch 8/10
39/39 [==============================] - 5s 125ms/step - loss: 0.4568 - binary_accuracy: 0.9984
Epoch 9/10
39/39 [==============================] - 5s 125ms/step - loss: 0.3911 - binary_accuracy: 0.9984
Epoch 10/10
39/39 [=================

In [14]:
test_loss, test_acc = model.evaluate(tf.convert_to_tensor(test["cleaned_text"]), tf.convert_to_tensor(test["Label_bias"].astype("category")), verbose=1)

10/10 [==============================] - 1s 11ms/step - loss: 1.9336 - binary_accuracy: 0.6806


In [17]:
export_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Activation('sigmoid')
])

export_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)


In [18]:
text_to_predict = ["It is the third time that Prime Minister Rishi Sunak has backed down in the face of rebellious backbenchers since taking power in October."]
text_to_predict = remove_stopwords(text_to_predict)
text_to_predict = lemmatization(text_to_predict)
text_to_predict = [' '.join(sentence) for sentence in text_to_predict]

predictions = export_model.predict(text_to_predict)
predictions

array([[0.5485208]], dtype=float32)